In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the data
train = pd.read_csv()
test = pd.read_csv(r"C:\Users\grace\Downloads\test_lAUu6dG.csv"r"C:\Users\grace\Downloads\train_ctrUa4K.csv")

train_original = train.copy()
test_original = test.copy()


In [3]:
# Check for missing values
print(train.isnull().sum())


Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [5]:
# Fill categorical columns with mode
for col in ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History', 'Loan_Amount_Term']:
    train[col] = train[col].fillna(train[col].mode()[0])
    test[col] = test[col].fillna(test[col].mode()[0])

train['LoanAmount'] = train['LoanAmount'].fillna(train['LoanAmount'].median())
test['LoanAmount'] = test['LoanAmount'].fillna(test['LoanAmount'].median())


# Confirm no missing values remain
print(train.isnull().sum())


Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64


In [13]:
train['Loan_Status'] = train['Loan_Status'].map({'Y': 1, 'N': 0})

# Concatenate train and test for consistent encoding
train['Source'] = 'train'
test['Source'] = 'test'
test['Loan_Status'] = np.nan  # Add column for consistent structure
combined = pd.concat([train, test])


In [7]:
combined = pd.get_dummies(combined, columns=['Gender', 'Married', 'Dependents', 'Education',
                                              'Self_Employed', 'Property_Area'], drop_first=True)


In [8]:
train = combined[combined['Source'] == 'train'].drop(['Source'], axis=1)
test = combined[combined['Source'] == 'test'].drop(['Source', 'Loan_Status'], axis=1)


split data

In [12]:

X = train.drop(['Loan_ID', 'Loan_Status'], axis=1)
y = train['Loan_Status']

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

Train Model

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train);


validate

In [15]:
y_pred = model.predict(X_val)
print("Validation Accuracy:", accuracy_score(y_val, y_pred))


Validation Accuracy: 0.7804878048780488


Predict on test set

In [17]:
X_test = test.drop(['Loan_ID'], axis=1)
X_test = X_test.reindex(columns=X.columns, fill_value=0)  # Align with training columns
pred_test = model.predict(X_test)


In [18]:
submission = pd.DataFrame({
    'Loan_ID': test['Loan_ID'],
    'Loan_Status': np.where(pred_test == 1, 'Y', 'N')
})

submission.to_csv(r"C:\Users\grace\Downloads\submission.csv", index=False)
print("Submission file saved as submission.csv")


Submission file saved as submission.csv


In [19]:
submission.head()

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,N
